# Operational Impact — Chicago Mobility Duration Model\n\nEste notebook traduce las métricas del modelo de duración de viajes a KPIs operativos (tiempo de espera estimado y ahorro potencial).\n\nLos números son ilustrativos y se basan en las métricas guardadas por `main.py` en `artifacts/metrics.json`.\n

In [ ]:
from pathlib import Path\nimport json\n\nimport pandas as pd\n\n# Cargar métricas guardadas por train_model (val/test)\nmetrics_path = Path('artifacts/metrics.json')\nif not metrics_path.exists():\n    raise FileNotFoundError('Ejecuta primero `make train` para generar artifacts/metrics.json')\n\nmetrics = json.loads(metrics_path.read_text())\nmetrics\n

In [ ]:
# Extraer métricas de test (MAE y RMSE)\ntest_metrics = metrics.get('test', {})\nmae_sec = float(test_metrics.get('mae', 0.0))\nrmse_sec = float(test_metrics.get('rmse', 0.0))\n\nprint(f'MAE test (segundos): {mae_sec:.2f}')\nprint(f'RMSE test (segundos): {rmse_sec:.2f}')\n

In [ ]:
# Supuestos de ejemplo para convertir errores en KPIs operativos\navg_trips_per_day = 200_000   # número de viajes diarios aproximado\nbaseline_mae_sec = mae_sec * 1.3  # baseline 30% peor que el modelo\n\n# Ahorro de segundos por viaje vs baseline\nsaved_seconds_per_trip = baseline_mae_sec - mae_sec\nsaved_minutes_per_trip = saved_seconds_per_trip / 60.0\n\n# Ahorro total de tiempo de viaje al día (en horas)\nsaved_hours_per_day = avg_trips_per_day * saved_seconds_per_trip / 3600.0\n\nprint(f'Ahorro medio por viaje vs baseline: {saved_seconds_per_trip:.1f} s ({saved_minutes_per_trip:.2f} min)')\nprint(f'Ahorro total estimado por día: {saved_hours_per_day:.1f} horas de tiempo de viaje')\n

## De tiempo de viaje a tiempo de espera\n\nSi asumimos que una parte del tiempo de viaje se refleja en el tiempo de espera percibido por el usuario (p.ej. congestión, reposicionamiento de flota),\npodemos hacer una aproximación simplificada del impacto en tiempo de espera.\n

In [ ]:
# Supuesto: una fracción del ahorro de tiempo de viaje se traduce en menos tiempo de espera.\nwait_time_conversion = 0.3  # 30% del ahorro de tiempo de viaje afecta tiempos de espera\nsaved_wait_hours_per_day = saved_hours_per_day * wait_time_conversion\n\nprint(f'Ahorro estimado en tiempo de espera: {saved_wait_hours_per_day:.1f} horas/día (supuesto 30%)')\n

### Notas\n\n- Este análisis es puramente ilustrativo y depende fuertemente de los supuestos (viajes/día, relación entre tiempo de viaje y de espera, calidad del baseline).\n- Para un caso real, estos supuestos deben validarse con datos de operaciones y SLAs definidos.\n- El objetivo del notebook es mostrar cómo conectar métricas de error del modelo con KPIs operativos comprensibles.\n